In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Image processing
import matplotlib.pyplot as plt
import seaborn as sns
import imageio
import skimage
import skimage.io
import skimage.transform
import math
import time
from imageio import imread

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras import optimizers


#Pre-trained model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19 
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.nasnet import NASNetLarge
from keras.applications.densenet import DenseNet169
from keras.models import Model
from keras.utils import plot_model
from keras import Input

Using TensorFlow backend.


In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 )

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            )

Found 1609 images belonging to 2 classes.
Found 674 images belonging to 2 classes.


In [0]:
PATH='/content/drive/My Drive'

In [0]:

train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'test')

In [0]:
train_melanoma_dir = os.path.join(train_dir, 'melanoma')  # directory with our training cat pictures
train_non_melanoma_dir = os.path.join(train_dir, 'non_melanoma')  # directory with our training dog pictures
test_melanoma_dir = os.path.join(test_dir, 'melanoma')  # directory with our validation cat pictures
test_non_melanoma_dir = os.path.join(test_dir, 'non_melanoma')  # directory with our validation dog pictures

In [0]:
num_melanoma_tr = len(os.listdir(train_melanoma_dir))
num_non_melanoma_tr = len(os.listdir(train_non_melanoma_dir))

num_melanoma_val = len(os.listdir(test_melanoma_dir))
num_non_melanoma_val = len(os.listdir(test_non_melanoma_dir))

total_train = num_melanoma_tr + num_non_melanoma_tr
total_test = num_melanoma_val + num_non_melanoma_val

In [8]:
print('total training melanoma images:', num_melanoma_tr)
print('total training non melanoma images:', num_non_melanoma_tr)

print('total validation melanoma images:', num_melanoma_val)
print('total validation non melanoma images:', num_non_melanoma_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_test)

total training melanoma images: 850
total training non melanoma images: 759
total validation melanoma images: 370
total validation non melanoma images: 304
--
Total training images: 1609
Total validation images: 674


In [9]:
image_batch_train, label_batch_train = next(iter(training_set))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(training_set.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 2)
['Melanoma' 'Non_Melanoma']


In [10]:
baseModel = Xception(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = Flatten(name = "flattern")(headModel)
headModel = Dense(512, activation = "relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0          

In [11]:
sgd = optimizers.SGD(lr=0.001, momentum=0.9)
rms = optimizers.RMSprop(0.001)
adam = optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Compiled!')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Compiled!


In [0]:
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

earlystopper1 = EarlyStopping(monitor='val_loss',mode = 'min', patience=10, verbose=1)

# Save the best model during the traning
checkpointer1 = ModelCheckpoint('best_model_Xception.h5'
                                ,monitor='val_acc' #'val_acc' for colab 
                                ,mode = 'max'
                                ,verbose=1
                                ,save_best_only=True
                                ,save_weights_only=True)

In [0]:
sample_training_images, _ = next(training_set)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import *


In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
import time
starttime=time.time()
training=model.fit_generator(
        training_set,
        #steps_per_epoch=100,
        epochs=20,
        validation_data = test_set, 
        #validation_steps = 30,
        callbacks=[earlystopper1, checkpointer1],
        shuffle= True
       )

Exetime = time.time()-starttime
print("Seconds: ", Exetime)
local_time = time.gmtime(Exetime)
print("Execution Time: ",time.strftime(" %H:%M:%S", local_time))



Epoch 1/20
51/51 [==============================] - 901s 18s/step - loss: 0.8884 - acc: 0.4955 - val_loss: 0.7440 - val_acc: 0.5371

Epoch 00001: val_acc improved from -inf to 0.53709, saving model to best_model_Xception.h5
Epoch 2/20
51/51 [==============================] - 84s 2s/step - loss: 0.6917 - acc: 0.5302 - val_loss: 0.7334 - val_acc: 0.5356

Epoch 00002: val_acc did not improve from 0.53709
Epoch 3/20
29/51 [================>.............] - ETA: 32s - loss: 0.6917 - acc: 0.5283

In [0]:
#Loading Saved model
model.load_weights('/content/best_model_Xception.h5')
y_prob = model.predict(test_set) 

In [0]:
predicted = y_prob.argmax(axis=-1)
actual = test_set.classes
class_labels = test_set.class_indices
print(predicted)
print(actual)
print(class_labels)

In [0]:
#Accuracy calculation and confusion matrix generation

from numpy import argmax
import pandas as pd
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
import seaborn as sn
import numpy as np
class_labels =['A','B'] 
predicted = y_prob.argmax(axis=-1)
actual = test_set.classes
results = confusion_matrix(actual, predicted)
acc = accuracy_score(actual, predicted) 
report = classification_report(actual, predicted)

print(results) 
print ('Accuracy Score :',acc) 
print ('Report :', report)


#Confusion matrix txt file
result_json_file = 'confmat.txt' 
with open(result_json_file, mode='w') as f:
    print(results,file=f)
print("confmat.txt Done!!")

#Report txt file
report_json_file = 'report.txt' 
with open(report_json_file, mode='w') as f:
  f.write(report)
print("report.txt Done!!")

#Heat Map draw
sn.set(font_scale=1.2)
plt.figure(figsize = (10,10))
map = sn.heatmap(results, annot=True,annot_kws={"size": 10}, center=1,cmap="Blues", fmt='.1f',lw=0.7, cbar=True, cbar_kws={'label': '# Images', 'orientation': 'horizontal'})
map.set_title('Confusion matrix')
map.set_xticklabels(class_labels, fontsize = 8)
map.set_yticklabels(class_labels, fontsize = 8)
figure = map.get_figure()  
figure.savefig("Heat_map.png")

In [0]:
#plot loss
fig = plt.figure(figsize=(16, 16)) 
plt.rcParams['figure.figsize'] = (8, 8)

plt.plot(training.history['loss'], label="Loss")
plt.plot(training.history['val_loss'], label="Validation loss")
plt.title('%s: loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
fig.savefig("loss_plot.png")

In [0]:
#plot model accuracy
fig1 = plt.figure(figsize=(16, 16)) 

plt.plot(training.history['acc'])
plt.plot(training.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
fig1.savefig("Accuracy_plot.png")